In [16]:
Dataset = "FoCus"                                  # Synthetic-PersonaChat, Blended Skill Talk, PEC, ConvAI2, FoCus, IT-ConvAI2
LLM_name = "Qwen2-7B-Instruct"                   # Mistral-7B-Instruct, Llama3-1-8B-Instruct, Qwen2-7B-Instruct

_COT = ""
# _COT = "-COT"

In [17]:
import pandas as pd

ds = pd.read_csv(f'Raw Responses/{Dataset}/{LLM_name}{_COT}.csv')

# ds = pd.read_csv(f'New Responses/{Dataset}/{LLM_name}{_COT}.csv')

print("Shape:", ds.shape)

print("\nMissing Values:\n", ds.isnull().sum())

ds

Shape: (1000, 2)

Missing Values:
 gen_response     0
response_time    0
dtype: int64


,gen_response,response_time
0,"{\n ""response"": ""Nazareth House is a historic...",3.733489
1,"{\n ""response"": ""This historic home in Jerich...",4.197150
2,"{\n ""response"": ""The Marion Palace Theatre is...",2.774832
3,"{\n ""response"": ""Technische Universität Darms...",4.265017
4,"{\n ""response"": ""Sure, I'd be delighted to he...",4.194262
...,...,...
995,"{\n ""response"": ""Sure, the museum/Pharmacy yo...",4.128956
996,"{\n ""response"": ""This ancient city, known as ...",2.974082
997,"{\n ""response"": ""Armagh County Museum was ope...",4.205184
998,"{\n ""response"": ""This place sounds amazing! I...",4.270991


In [18]:
print(ds.iloc[167]["gen_response"])

{
  "response": "Thursday Island Cemetery is historically significant as it commemorates the lives of Japanese pearl divers who played a crucial role in the industry. Many of these divers lost their lives while working in dangerous conditions. The cemetery not only serves as a poignant reminder of their sacrifices but also highlights the cultural exchange and economic ties between Japan and Australia during that time. As someone who appreciates cemeteries and has an interest in Japanese culture, you would find this place both intriguing and emotionally moving. Additionally, being a heritage site, it offers


In [19]:
import re
import json
import pandas as pd

# Define functions
def find_first_valid_json(text):

    
    json_objects = re.findall(r'\{.*?\}', text, re.DOTALL)
    for obj in json_objects:
        try:
            json_obj = json.loads(obj)
            if "response" in json_obj:  # Only check for "response"
                return json_obj
        except json.JSONDecodeError:
            continue
    return None

def get_response(text):
    if text is not None:  # Check if text is not None
        try:
            return text['response']
        except (ValueError, SyntaxError, KeyError):
            return None
    return None



# Replace empty strings in 'gen_response' with None
ds.loc[ds['gen_response'] == '', 'gen_response'] = None

# Apply the find_first_valid_json function
ds['gen_response'] = ds['gen_response'].apply(lambda x: find_first_valid_json(x))

# Convert gen_response to None if it's not a valid string
ds['gen_response'] = ds['gen_response'].apply(lambda x: None if pd.isna(x) or x == 'nan' or isinstance(x, float) else x)

# Extract 'response' from the JSON objects
ds['gen_response'] = ds['gen_response'].apply(lambda x: get_response(x))

# Keep the 'response_time' column unchanged
ds['response_time'] = ds['response_time']

# Define the new column order
new_column_order = ['gen_response', 'response_time']

# Reorder the columns
ds = ds[new_column_order]

print("\nMissing Values:\n", ds.isnull().sum())
print(ds.shape)
ds



Missing Values:
 gen_response     509
response_time      0
dtype: int64
(1000, 2)


,gen_response,response_time
0,Nazareth House is a historic place located on ...,3.733489
1,None,4.197150
2,The Marion Palace Theatre is a stunning exampl...,2.774832
3,None,4.265017
4,None,4.194262
...,...,...
995,"Sure, the museum/Pharmacy you're referring to ...",4.128956
996,"This ancient city, known as Pundranagara or Pa...",2.974082
997,None,4.205184
998,None,4.270991


In [20]:
ds.to_csv(f'Responses/{Dataset}/{LLM_name}{_COT}.csv', index=False)